## Step 1: Generating PES using classical methods

Our task is to investigate the ground-state problem which is to determine the ground-state wavefunction and energy associated with a system of interest. We start by discussing several classical computational methods of determining the electronic ground-state structure. 

#### Variational approaches:

The variational approach in quantum mechanics is a method of approximating the ground-state wavefunction. An initial guess is made for the wavefunction which depends on several parameters. The expectation value of the energy is calculated using this trial wavefunction, and the parameters are varied such that the energy is minimized. The result corresponds to the ground-state wavefunction and the associated energy represents an upper bound to the ground state energy.

Here, we consider the performance of two variational methods: The Hartree-Fock (HF) method and the Configuration interaction (CI) method. In the HF method, the wavefunction used is assumed to the product of single-electron wavefunctions and is subject to the antisymmetric requirement [1]. The many-body problem is simplified by considering each electron to move in an effective potential which represents the average density of all the other electrons. A strength of the HF method can be seen in the simplicity of the theory, however, the simplification also leads to low accuracy of the ground-state energy because electron correlation is not included. CI methods aim to overcome this issue by considering multiple orbital configurations of the electrons, whereas the HF method only considers a single configuration. In this way, spatial correlations of the electrons are included and the resulting energies are more accurate. While CI methods are generally more accurate than the HF method, a downside is that commonly used approximate CI methods do not necessarily scale correctly with the number of electrons in the system. To be specific, the energy should be linearly proptional to the number of electrons in the system, however, CI methods do not guarantee that this relationship will hold.

An non-variational approach, known as the Coupled Cluster (CC) method, guarantees the correct scaling of the system with the number of electrons. The CC method considers multiple electron configurations and is developped using perturbation theory. A significant disadvantage associated with the CC method is that the calculated energy does not provide an upper-bound for the exact energy.

To see these methods in practice, we analyzed the following set of molecules:
- $H_2$
- $LiH$
- $H_4$
- $H_2 O$
- $N_2$

We investigated the energy of each molecule as a function of the bond length using the HF method, the Coupled Cluster Singles and Doubles (CCSD), and the exact answer within the chosen basis Full Configuration Interaction (FCI). Please see the S1_Classical_Methods notebook for our calculation details. The results of our calculations are as follows.

![classical_methods](./classical_methods.png)

The above plots show that for every molecule, the exact energy is lower than the energy calculated using the HF method. This was expected since variational methods such as the HF method provide an upper bound for the exact energy. We also see that the CCSD energy can be lower than the exact energy, which is consistent with our understanding that the CCSD does not provide an upper-bound.

Another observation is that for all molecules, there are significant discrepancies between the approximate and exact energies- particularly in the region where the bond is longer than the equilibrium length. We can see that the energy of the weakly correlated molecules on the bottom row ($H_2$ and $LiH$) are better behaved than the energies associated with the highly correlated molecules on the top row ($H_2 O$, $N_2$ and $H_4$). In particular, the relationship between the CCSD energy and bond length exhibit unexpected behaviours for the $N_2$ and $H_2 O$ molecules.

Depending on what properties are to be calculated, the runtime and accuracy of the calculation are both factors which dictate the ideal method to be used. The exact solution scales exponentially with the size of the system, and approximate methods scale polynomially with the size. None of the methods are ideal solutions to the ground-state problem for every application. We now turn to the quantum world to see if we can overcome some of these limitations.

## Step 2: Generating the qubit Hamiltonian

To proceed to VQE one needs to generate the qubit Hamiltonian, the easiest path is via first generating the electronic Hamiltonian in the second quantized form and then transform it into the qubit form using one of the fermion-to-qubit transformation: Jordan-Wigner or Bravyi-Kitaev.  Next, some qubit operators can be substituted by number ($\pm 1$) because their states are stationary for the specific electronic state (e.g., ground state). This reduction is very useful for fitting larger problem in a fewer qubit description and is based on Hamiltonian symmetries.

Please see the S2_Hamiltonian_gen notebook for calculation details. The notebook demonstrates the Hamiltonian generation for h2 considering the equilibrium bond length of $0.74\overset{\circ}{A}$. The fermionic Hamiltonian is mapped to the qubit (spin) Hamiltonian using the Jordan-Wigner transformation. The ground state energy of the fermionic Hamiltonian ($-1.12 \, E_h$) is compared to that of the qubit Hamiltonian ($-1.14 \, E_h$), and also the energy of a smaller Hamiltonian generated by the qubit-tapering technique ($-1.10 \, E_h$). All three values agree with one-another to an accuracy of 0.1 eV.

#### 2.1 What are the requirements for a function of qubit operators to be a valid mapping for the fermionic operators?

First, recall that fermionic creation/annihilation operators in the second quantization formalism are required to obey the anti-commutation relations:
$$ \{  \hat{a}_p, \hat{a}_q^\dagger \} = \delta_{pq} \qquad \{  \hat{a}_p^\dagger, \hat{a}_q^\dagger \} = \{  \hat{a}_p, \hat{a}_q \} = 0 $$

To run our quantum algorithm, we first must transform the Hamiltonian, which is originally expressed in second quantization, as a linear combination of the Pauli operators. In order for the mapping transformation to be valid, we must ensure that it preserves the anti-commutation of the fermionic operators [2]. This is not a given, as Pauli operators do not naturally obey the anti-commutation relations.

To illustrate the transformation, we consider the Jordan-Wigner transformation. Let $|0>_j$ ($|1>_j$) denote the unoccupied (occupied) state of the $j$-th spin orbital. For a single operator, the creation operators could be expressed as:
$$ \hat{a}^\dagger \rightarrow |1><0|_j = \begin{pmatrix} 0 & 0\\ 1 & 0 \end{pmatrix} = \frac{X_j - i Y_j}{2} $$
The problem with this transformation is that the Pauli operators do not satisfy the anti-commutation relation.

There is an easy and intuitive way to change the transformation so that the anti-commutation relation is enforced. We apply Z gates to each qubit from $0$ to $j-1$.
$$ \hat{a}^\dagger \rightarrow \frac{X_j - i Y_j}{2} \otimes Z_0 \otimes \dots \otimes Z_{j-1} $$
One can easily verify that the anti-commutation relation is now enforced.

#### 2.2 The electronic Hamiltonian is real (due to time-reversal symmetry), what consequences does that have on the terms in the qubit Hamiltonian after the Jordan-Wigner transformation?

Time reversal ensures that the fermionic Hamiltonian is real and symmetric. By implication, the Jordan-Wigner transformed qubit Hamiltonian must also be real. If the mapping is performed such that the resulting operators are expressed as products of the Pauli matrices [3] then the assertion that the Hamiltonian be real implies that operators composed of an even number of Y gates are not included. This reduces the set of possible operators, reducing the depth of the circuit.

## References

[1] Hoggan, Philip E. Novel Electronic Structure Theory: General Innovations and Strongly Correlated Systems. Academic Press, 2018.

[2] Tilly, Jules, et al. "The variational quantum eigensolver: a review of methods and best practices." arXiv preprint arXiv:2111.05176 (2021).

[3] Tang, Ho Lun, et al. "qubit-adapt-vqe: An adaptive algorithm for constructing hardware-efficient ansätze on a quantum processor." PRX Quantum 2.2 (2021): 020310.